Python notebook developed by:
Rodrigo Cedeno & Emanuele Capizzi
For project ...

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
import matplotlib.figure
import fiona.crs
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
import xarray as xr
import time
import yaml
import uuid
import os
from glob import iglob # data access in file manager
import rasterio
from os.path import join # same
import plotly.graph_objects as go
import rioxarray as rio

#Hide Warnings
import warnings
warnings.filterwarnings('ignore')

#Progress bar
import ipywidgets as widgets
from termcolor import colored # Print colored text

In [2]:
file_basepath = '/home/asi/datacube_conf/data/meteo/climami'
#search inside sub folders
list_files = sorted(list(iglob(join(file_basepath, '*.tif*'), recursive=True)))
#without searching inside sub folders
#list_files = sorted(list(iglob(join(files_to_move, '*.tif'), recursive=True)))
list_files

['/home/asi/datacube_conf/data/meteo/climami/raster_massima_ESTATE_UHI_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_ESTATE_UHI_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_INVERNO_UHI_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_INVERNO_UHI_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_ONDATA_DI_CALORE_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_ONDATA_DI_CALORE_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_minima_INVERNO_UHI_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_minima_INVERNO_UHI_2200.tif']

In [3]:
file_names = []
for path in list_files:
    # Extract the last portion after the last slash (/)
    file_name = path.split('/')[-1]
    # Remove the '_30m.tif' suffix from the file name
    file_name = file_name.replace('.tif', '')
    file_names.append(file_name)
print(file_names)

['raster_massima_ESTATE_UHI_2200', 'raster_massima_ONDATA_DI_CALORE_1100', 'raster_massima_ONDATA_DI_CALORE_2200', 'raster_media_ESTATE_UHI_2200', 'raster_media_INVERNO_UHI_1100', 'raster_media_INVERNO_UHI_2200', 'raster_media_ONDATA_DI_CALORE_1100', 'raster_media_ONDATA_DI_CALORE_2200', 'raster_minima_INVERNO_UHI_1100', 'raster_minima_INVERNO_UHI_2200']


In [4]:
file_dict = {}
for key in file_names:
    for value in list_files:
        file_dict[key] = value
        list_files.remove(value)
        break
file_dict

{'raster_massima_ESTATE_UHI_2200': '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ESTATE_UHI_2200.tif',
 'raster_massima_ONDATA_DI_CALORE_1100': '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_1100.tif',
 'raster_massima_ONDATA_DI_CALORE_2200': '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_2200.tif',
 'raster_media_ESTATE_UHI_2200': '/home/asi/datacube_conf/data/meteo/climami/raster_media_ESTATE_UHI_2200.tif',
 'raster_media_INVERNO_UHI_1100': '/home/asi/datacube_conf/data/meteo/climami/raster_media_INVERNO_UHI_1100.tif',
 'raster_media_INVERNO_UHI_2200': '/home/asi/datacube_conf/data/meteo/climami/raster_media_INVERNO_UHI_2200.tif',
 'raster_media_ONDATA_DI_CALORE_1100': '/home/asi/datacube_conf/data/meteo/climami/raster_media_ONDATA_DI_CALORE_1100.tif',
 'raster_media_ONDATA_DI_CALORE_2200': '/home/asi/datacube_conf/data/meteo/climami/raster_media_ONDATA_DI_CALORE_2200.tif',
 'raster_minima_INVERNO_UHI_1100': '/hom

In [5]:
list_files = sorted(list(iglob(join(file_basepath, '*.tif*'), recursive=True)))
#without searching inside sub folders
#list_files = sorted(list(iglob(join(files_to_move, '*.tif'), recursive=True)))
list_files

['/home/asi/datacube_conf/data/meteo/climami/raster_massima_ESTATE_UHI_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_ESTATE_UHI_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_INVERNO_UHI_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_INVERNO_UHI_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_ONDATA_DI_CALORE_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_media_ONDATA_DI_CALORE_2200.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_minima_INVERNO_UHI_1100.tif',
 '/home/asi/datacube_conf/data/meteo/climami/raster_minima_INVERNO_UHI_2200.tif']

# Generate product measurements list

In [6]:
f = widgets.IntProgress(min=0, max=1, description='Processing images', bar_style='info')
display(f)
p_name = "climami"
text_p0 = """name: """ + p_name + """
description: raster data retrived from CLIMAMI Project (Osservatorio Meteorologico Milano).
metadata_type: eo3

metadata:
    product:
        name: """ + p_name + """
        format: GeoTIFF

storage:
    crs: EPSG:32632
    dimension_order: [time, x, y]
    resolution: 
        x: 30 
        y: -30"""

file = list_files[0]
date_uuid = file[file.find('202'):file.find('202')+8]
src = rasterio.open(file) 
xmin,ymin,xmax,ymax = src.bounds
affine = src.transform
shape = src.shape
text_p2_1 = """"""
text_p2 = """"""
text_p1 = ("""

measurements:""")
""""""
for band in file_names:
    text_p2_1 = """
    - name: """+band+"""
      dtype: float32
      nodata: NaN
      units: ''
"""
    text_p2 = text_p2 + text_p2_1
text_p3 = text_p0 + text_p1 + text_p2
yaml_text = open('/home/asi/datacube_conf/'+p_name+'.yaml', "wt")
yaml_text.write(text_p3)
yaml_text.close()
os.system("""cd /home/asi/datacube_conf; 
datacube product add /home/asi/datacube_conf/"""+p_name+""".yaml""")
#os.system("""cd /home/asi/datacube_conf; 
#datacube product update --allow-unsafe /home/asi/datacube_conf/"""+p_name+""".yaml""")
f.value += 1
        
print(colored('All Climami products have been updated into the ODC','green'))

IntProgress(value=0, bar_style='info', description='Processing images', max=1)

Adding "climami" (this might take a while) DONE
All Climami products have been updated into the ODC


## Generate and Register Dataset 

In [7]:
f = widgets.IntProgress(min=0, max=len(list_files), description='Processing images', bar_style='info')
display(f)

for file in list_files:
    filename='/home/asi/datacube_conf/data/meteo/climami/climami.yaml'
    date_uuid = '20230209'
    print(file)
    src = rasterio.open(file)
    xmin,ymin,xmax,ymax = src.bounds
    affine = src.transform
    shape = src.shape
    text_p1 = ("""$schema: https://schemas.opendatacube.org/dataset

id: 50000000-0000-0000-0000-0000"""+ str(date_uuid) +"""

product:
  name: climami
  href: https://www.progettoclimami.it/
  format: GeoTIFF

crs: EPSG:32632

geometry:
  type: Polygon
  coordinates: [[["""+str(xmin)+""", """+str(ymin)+"""], ["""+str(xmin)+""", """+str(ymax)+"""], ["""+str(xmax)+""", """+str(ymax)+"""], ["""+str(xmax)+""", """+str(ymin)+"""], ["""+str(xmin)+""", """+str(ymin)+"""]]]

grids:
  default:
    shape: ["""+str(shape[0])+""","""+str(shape[1])+"""]
    transform: ["""+str(affine[0])+""","""+str(affine[1])+""","""+str(affine[2])+""","""+str(affine[3])+""","""+str(affine[4])+""","""+str(affine[5])+""", 0,0,1]

lineage: {}
measurements:
""")
    text_p2 = """"""
    for band in file_names:
        text_p2_1 = """
    """+str(band)+""":
        path: """+str(file_dict[band])+""""""
        text_p2 = text_p2 + text_p2_1
    
    text_p3 = """
properties:
  odc:file_format: GeoTIFF
  datetime: """+str(date_uuid)+"""T10:30:00Z
    """
    yaml_text = open(filename.replace('.tif', '.yaml'), "wt")
    yaml_text.write(text_p1+text_p2+text_p3)
    yaml_text.close()
    os.system("""cd /home/asi/datacube_conf; 
    datacube dataset add """+filename.replace('.tif', '.yaml')) 

f.value += 1
        
print(colored('All Climami products have been imported into the ODC','green'))

IntProgress(value=0, bar_style='info', description='Processing images', max=10)

/home/asi/datacube_conf/data/meteo/climami/raster_massima_ESTATE_UHI_2200.tif
/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_1100.tif


2024-02-10 11:00:18,902 495 datacube.index.postgres._datasets WARNING Dataset 50000000-0000-0000-0000-000020230209 is already in the database


/home/asi/datacube_conf/data/meteo/climami/raster_massima_ONDATA_DI_CALORE_2200.tif


2024-02-10 11:00:26,292 516 datacube.index.postgres._datasets WARNING Dataset 50000000-0000-0000-0000-000020230209 is already in the database


...

/home/asi/datacube_conf/data/meteo/climami/raster_minima_INVERNO_UHI_1100.tif


2024-02-10 11:01:11,808 642 datacube.index.postgres._datasets WARNING Dataset 50000000-0000-0000-0000-000020230209 is already in the database


/home/asi/datacube_conf/data/meteo/climami/raster_minima_INVERNO_UHI_2200.tif


2024-02-10 11:01:19,170 663 datacube.index.postgres._datasets WARNING Dataset 50000000-0000-0000-0000-000020230209 is already in the database


All Climami products have been imported into the ODC
